# Semi-automatic Blood Masking

In [ ]:
import runpy
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from pathlib import Path

## Specify Dataset

In [ ]:
paths = runpy.run_path('../../params/paths.py')
INPUT_PATH = Path(paths['OUTPUT_BASE_PATH'], 'compare', 'invivo', 'voltage_HPC2')
DATASET_NAME = '02_02'

## Compute Per-pixel Temporal Fourier Transform

In [ ]:
input_file = Path(INPUT_PATH, DATASET_NAME, 'motion_corrected.tif')
data = tiff.imread(input_file).astype(float)
ft = np.absolute(np.fft.fft(data, axis=0))

### Plot Spectrum of Center Pixel

In [ ]:
plt.figure()
plt.plot(ft[:, ft.shape[1]//2, ft.shape[2]//2])
plt.ylabel('Amplitude')
plt.xlabel('Frequency')
plt.show()

## Compute Blood Image Candidates
* Left: Sum of low frequency band
* Center: Sum of high frequency band
* Right: Difference of the two

In [ ]:
MIN_FREQ = 10
MAX_FREQ = len(ft) // 2  # Nyquist
FREQ_STEP = 10
FREQ_END = 300 # examine up to this frequency
for mid_freq in range(MIN_FREQ + FREQ_STEP, FREQ_END, FREQ_STEP):
    print('[%d, %d] vs [%d, %d]' % (MIN_FREQ, mid_freq, mid_freq, MAX_FREQ))

    low_band = np.sum(ft[MIN_FREQ:mid_freq], axis=0)
    high_band = np.sum(ft[mid_freq:MAX_FREQ], axis=0)

    low_band_norm = low_band / np.amax(low_band)
    high_band_norm = high_band / np.amax(high_band)
    blood_img = np.maximum(high_band_norm - low_band_norm, 0)

    plt.figure(figsize=(17, 5))
    plt.subplot(1, 3, 1)
    plt.axis('off')
    plt.imshow(low_band, cmap='gray')
    plt.subplot(1, 3, 2)
    plt.axis('off')
    plt.imshow(high_band, cmap='gray')
    plt.subplot(1, 3, 3)
    plt.axis('off')
    plt.imshow(blood_img, cmap='gray')
    plt.show()

## Pick The Right One and Save The Mask

In [ ]:
MID_FREQ = 200 # specify the number that yielded a good blood image above
THRESHOLD = 0.08

low_band = np.sum(ft[MIN_FREQ:MID_FREQ], axis=0)
high_band = np.sum(ft[MID_FREQ:MAX_FREQ], axis=0)

low_band_norm = low_band / np.amax(low_band)
high_band_norm = high_band / np.amax(high_band)
blood_img = np.maximum(high_band_norm - low_band_norm, 0)

blood_mask = blood_img < THRESHOLD

plt.figure(figsize=(11, 5))
plt.subplot(1, 2, 1)
plt.axis('off')
plt.imshow(high_band_norm, cmap='gray')
plt.subplot(1, 2, 2)
plt.axis('off')
plt.imshow(blood_mask, cmap='gray')
plt.show()

tiff.imwrite(input_file.with_name('bloodmask.tif'), blood_mask, photometric='minisblack')